# 1-1. 하스스톤 전설 유저수 변화 데이터 크롤링
<pre>
▶ 기간 : 2022년 10월 ~ 2023년 11월 28일자
▶ 대상 : 현재 운영 중인 아시아, 유럽, 아메리카 서버에 존재하는 전설 등급 유저
▶ 사이트 : [순위표 - 하스스톤](https://hearthstone.blizzard.com/ko-kr/community/leaderboards)
</pre>

## 라이브러리

In [3]:
import requests # HTTP 프로토콜을 이용하여 웹사이트로부터 데이터를 송수신 받기 위한 라이브러리
from bs4 import BeautifulSoup # 웹페이지의 HTML, XML 파일에서 데이터를 추출하는 라이브러리
import openpyxl # 엑셀파일 작업을 위한 라이브러리
import pandas as pd # 구조화된 데이터나 표 형식의 데이터 분석 및 조작을 위한 라이브러리

# 웹페이지를 자동으로 조작하기 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager # 웹브라우저의 드라이버를 자동으로 관리하는 라이브러리

# Selenium에서 사용되는 웹요소를 찾기 위한 기능을 제공해주는 라이브러리
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re  #  정규표현식을 사용해 숫자를 추출하기 위한 라이브러리
from time import sleep  # 실행을 일시적으로 멈춰 로딩을 기다리기 위한 라이브러리

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


## 데이터 크롤링 함수

In [2]:
def scrape_hs_legend_data():

    # 엑셀 만들기
    wb = openpyxl.Workbook()

    # 워크시트 만들기
    ws = wb.active
    ws.title = '하스스톤 전설등급 유저수'

    # 데이터 추가하기
    ws.append(['시즌', '년도', '월', '서버', '유저수'])

    # 엑셀 저장하기
    wb.save('./hs_legend.xlsx')

    # 서버, 시즌 변수 정의
    servers = ['AP', 'US', 'EU']  # 아시아태평양,북미,유럽
    start_year = 2022 # 시작 시즌 년도
    start_month = 10  # 시작 시즌 월
    num_seasons = 14  # 121시즌까지의 총 개수

    # 엑셀 열기
    wb = openpyxl.load_workbook('./hs_legend.xlsx')
    ws = wb.active

    # 데이터를 저장할 리스트 생성
    data = []

    # 크롬 설정
    chrome_options = Options() # 크롬의 설정을 조작하기 위한 객체 생성
    chrome_options.add_experimental_option("detach", True) # 크롬을 실행 후, 스크립트가 종료되더라도 브라우저가 계속 실행되도록 설정
    chrome_options.add_argument('--no-sandbox') # 브라우저 보안으로 인해 샌드박스 모드에서 실행되지 않도록 설정
    chrome_options.add_argument('--disable-dev-shm-usage') # /dev/shm - 공유메모리 사용, /dev/shm을 사용하지 않도록 설정해서 메모리 부족을 회피

    # 108~121 시즌까지 데이터 수집하도록 루프
    for season in range(108, 108 + num_seasons):
        # 서버별로 URL을 다르게 접속하는 루프
        for server in servers:

            # ChromeDriverManager를 사용하여 크롬 드라이버를 초기화.
            driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

            # URL 열기
            url = f"https://hearthstone.blizzard.com/ko-kr/community/leaderboards?region={server}&seasonId={season}"
            driver.get(url)

            # 엘리먼트가 나타날 때까지 대기
            try:  
                element_present = EC.presence_of_element_located((By.CLASS_NAME, 'metadata')) # metadata 엘리먼트 찾는 조건을 설정하고 객체를 생성
                WebDriverWait(driver, 10).until(element_present) # 최대 10초 대기
            except Exception as e: # 에외 처리
                print(f"엘리먼트 대기 중 예외 발생: {e}")
                driver.quit()
                continue

            # BeautifulSoup을 사용하여 HTML 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # metadata 엘리먼트 선택
            metadata = soup.select_one('.metadata')

            # 만약 metadata가 존재한다면 숫자를 추출하고 워크시트와 데이터 리스트에 추가
            if metadata:
                num_text = metadata.text # metadata에서 텍스트 추출
                num_match = re.search(r'\d+', num_text) # 숫자 추출

                # num_match가 None이 아닌지 확인 후에 처리
                if num_match:
                    num = num_match.group() # num_match에서 숫자추출한 부분을 반환

                    # '년도'와 '월' 데이터 추가
                    ws.append([season, start_year, start_month, server, num])

                    # 데이터를 리스트에 추가
                    data.append([season, start_year, start_month, server, num])
                else:
                    print(f"시즌 {season}, 서버 {server}에서 유저 수를 찾을 수 없습니다.")
           
            # Chrome 드라이버 종료
            driver.quit()

        # 다음 시즌의 시작년도와 시작월 설정
        start_month += 1
        if start_month > 12:
            start_month = 1
            start_year += 1

    # .xlsx 형식으로 엑셀 저장
    wb.save('./hs_legend.xlsx')

    # 데이터 리스트 반환
    return data

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


## 실행 코드 및 결과 .csv 저장

In [3]:
# 함수 호출하여 데이터 수집
scraped_data = scrape_hs_legend_data()

# 데이터를 포함한 리스트에서 DataFrame을 생성
save_data = pd.DataFrame(scraped_data, columns=['Season', 'Year', 'Month', 'Server', 'Number'])

# DataFrame을 UTF-8 인코딩으로 CSV 파일로 저장
save_data.to_csv('./hs_legend.csv', encoding='utf-8', index=False)

# 실행을 완료했다는 출력
print("Data Crawling Complete")

Data Crawling Complete
